In [ ]:
## Part 2 Instructions: Have Customers Narrow Their Travel Searches Based on Temperature and Precipitation

In [1]:
import pandas as pd
import gmaps
import requests
from config import g_key

In [2]:
# Read Files
city_data_df = pd.read_csv("weather_data/WeatherPy_challenge.csv")
city_data_df.dtypes

Unnamed: 0                    int64
City                         object
Country                      object
Date                         object
Lat                         float64
Lng                         float64
Max Temp                    float64
Humidity                      int64
Cloudiness                    int64
Wind Speed                  float64
Current Description          object
Rain inches (last 3 hrs)    float64
Snow inches (last 3 hrs)    float64
dtype: object

In [3]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain = str(input("Do you want it to be raining? (yes/no) "))
snow = str(input("Do you want it to be snowing? (yes/no) "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) yes
Do you want it to be snowing? (yes/no) no


In [4]:
# Filter the dataset to find the cities that fit the criteria.
preferred_temp_city_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

if (rain == "no") & (snow == "no"):
    preferred_cities_df = preferred_temp_city_df.loc[(city_data_df["Rain inches (last 3 hrs)"] == 0) &
                                                     (city_data_df["Snow inches (last 3 hrs)"] == 0)]
elif (rain == "yes") & (snow == "no"):
    preferred_cities_df = preferred_temp_city_df.loc[(city_data_df["Rain inches (last 3 hrs)"] != 0) &
                                                     (city_data_df["Snow inches (last 3 hrs)"] == 0)]
elif (rain == "no") & (snow == "yes"):
    preferred_cities_df = preferred_temp_city_df.loc[(city_data_df["Rain inches (last 3 hrs)"] == 0) &
                                                     (city_data_df["Snow inches (last 3 hrs)"] != 0)]
else:
    preferred_cities_df = preferred_temp_city_df.loc[(city_data_df["Rain inches (last 3 hrs)"] != 0) &
                                                     (city_data_df["Snow inches (last 3 hrs)"] != 0)]

    
preferred_cities_df.count()

Unnamed: 0                  29
City                        29
Country                     29
Date                        29
Lat                         29
Lng                         29
Max Temp                    29
Humidity                    29
Cloudiness                  29
Wind Speed                  29
Current Description         29
Rain inches (last 3 hrs)    29
Snow inches (last 3 hrs)    29
dtype: int64

In [5]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df.copy()[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""

In [6]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [8]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
17,Muli,MV,82.89,light rain,2.92,73.57,Medhufushi Island Resort
65,Puerto Ayora,EC,78.01,light rain,-0.74,-90.35,Finch Bay Galapagos Hotel
68,Luwuk,ID,85.17,light rain,-0.95,122.79,Hotel Santika Luwuk
84,Dingle,PH,81.34,light rain,11.00,122.67,Camp Pasica
93,Butaritari,KI,84.38,light rain,3.07,172.79,Isles Sunset Lodge
98,Meulaboh,ID,81.50,light rain,4.14,96.13,Kost Pria
109,Avarua,CK,84.20,light rain,-21.21,-159.78,Paradise Inn
119,Ballina,AU,77.00,light rain,-28.87,153.57,Ramada Hotel & Suites by Wyndham Ballina Byron
127,Gamba,GA,78.60,moderate rain,-2.65,10.00,
155,Kramat,ID,80.44,light rain,-8.13,113.45,Padepokan HM Arum Sabil


In [12]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file)
#city_data_df.to_csv(output_data_file, index_label="City_ID")

In [43]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [70]:
# Add vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))